# Финальный проект: вариант 2
## Задание 2. SQL
##  Anastasiia Sibiriakova (a_sibiriakova)
###  Поток 15.08

###  С корректировкой от 31.01.2023

In [3]:
import pandahouse as ph

In [4]:
connection = dict(database='default',
                  host='http://clickhouse.lab.karpov.courses:80',
                  user='student',
                  password='dpo_python_2020')

***

### 2.1 Очень усердные ученики
Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.

NB! Под очень усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

#### Решение
В переменную max_date сохраняем запрос с выбором максимальной даты  
    - with max_date as ( select toStartOfMonth(max(timest)) as max_date from default.peas )
Делаем запрос с выборкой 
    - select st_id, sum(correct) as count_correct_peas  from default.peas
с условием по максимальной дате из переменной max_date 
    - where toStartOfMonth(timest) = (select max_date from max_date)  
и группировкой по st_id 
    - group by st_id
на сгруппированные данные накладываем условие 
    - having count_correct_peas >= 20
оборачиваем это в подзапрос и считаем количество
    - select count(st_id) as count_best_students from ( подзапрос ) as best_students

In [5]:
query_task1 = ' with max_date as ' \
    ' ( select toStartOfMonth(max(timest)) as max_date from default.peas ) ' \
    ' select count(st_id) as count_best_students from ' \
    ' ( ' \
    ' select st_id, sum(correct) as count_correct_peas  from default.peas' \
    ' where toStartOfMonth(timest) = (select max_date from max_date)' \
    ' group by st_id' \
    ' having count_correct_peas >= 20' \
    ' ) as best_students' 

In [6]:
df_task1 = ph.read_clickhouse(query_task1, connection=connection)
df_task1.head()

,count_best_students
0,136


***

### 2.2 Оптимизация воронки
Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- CR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
- ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

#### Решение 
Чтобы посчитать для каждого студента количество верных горошин делаем запрос к таблице default.peas с группировкой по полю st_id     
Считаем суммы sum(correct) по всем правильным ответам и по правильным ответам по математике, для этого используем фильтр с условием sum(correct) filter (where subject == 'Math')  
В итоге получаем вот такой запрос:
    - select  st_id, 
        sum(correct) as correct_all,
        sum(correct) filter (where subject == 'Math') as correct_math 
      from default.peas 
      group by st_id 
Оборачиваем его в подзапрос, он нам пригодится
    - as count_peas
Объединяем таблицы с данными по студентам default.studs, наш подзапрос с количеством верных горошин count_peas и таблицу с данными по оплатам default.final_project_check методом left join объединяем по полю st_id  
В итоге получаем вот такой запрос:
    - select  default.studs.st_id as st_id, 
        test_grp, 
        correct_all, 
        correct_math, 
        subject, 
        sale_time, 
        money 
      from default.studs 
      left join 
      ( 
        select  st_id, 
          sum(correct) as correct_all,
          sum(correct) filter (where subject == 'Math') as correct_math 
        from default.peas 
        group by st_id 
      ) as count_peas 
      on default.studs.st_id = count_peas.st_id 
      left join default.final_project_check 
      on default.studs.st_id = default.final_project_check.st_id 
Сохраняем все данные в переменную all_data  
    - with all_data as 
                ( 
                  ...
                ) 

Пишем запрос для расчета метрик к нашей таблице со всеми данными, сохраненной в переменную all_data, группируем по группе test_grp

**ARPU** считаем как сумму всех оплат деленную на число уникальных студентов, так как студенты могут купить несколько курсов
    - round(sum(money) / count(distinct st_id), 0)
**ARPAU** считаем как сумму всех оплат деленную на число уникальных студентов с фильтром where correct_all > 10, так как по условию активным считается студент, за все время решивший больше 10 задач правильно в любых дисциплинах
    - round(sum(money) / count(distinct st_id) filter (where correct_all > 10), 0)
**CR** считаем как количество уникальных студентов с фильтром where money > 0, то есть платящие студенты, деленные на количество уникальных студентов в группе
    - round(count(distinct st_id) filter (where money > 0) / count(distinct st_id), 2)
**CR_active** считаем как количество уникальных студентов с фильтром where money > 0, то есть платящие студенты, деленные на количество уникальных активных студентов в группе с фильтром where correct_all > 10, так как по условию активным считается студент, за все время решивший больше 10 задач правильно в любых дисциплинах
    - round(count(distinct st_id) filter (where money > 0) / count(distinct st_id) filter (where correct_all > 10), 2)
**CR_math** считаем как количество уникальных студентов с фильтром where subject == 'Math', деленные на количество уникальных активных по математике студентов в группе с фильтром where correct_math >= 2, так как по условию активным по математике считается студент, за все время решивший 2 или больше задач правильно по математике
    - round(count(distinct st_id) filter (where money > 0 and subject == 'Math') / count(distinct st_id) filter (where correct_math >= 2), 2)

В итоге получаем вот такой запрос:
    - select  test_grp,      
        round(count(distinct st_id) filter (where money > 0) / count(distinct st_id), 2) as CR,   
        round(count(distinct st_id) filter (where money > 0) / count(distinct st_id) filter (where correct_all > 10), 2) as CR_active,   
        round(count(distinct st_id) filter (where money > 0 and subject == 'Math') / count(distinct st_id) filter (where correct_math >= 2), 2) as CR_math   
      from all_data   
      group by test_grp  

In [7]:
query_task2 = ' with all_data as' \
    ' (' \
    ' select  default.studs.st_id as st_id,' \
    ' test_grp,' \
    ' correct_all,' \
    ' correct_math,' \
    ' subject,' \
    ' sale_time,' \
    ' money' \
    ' from default.studs' \
    ' left join' \
    ' (' \
    ' select  st_id,' \
    ' sum(correct) as correct_all,' \
    ' sum(correct) filter (where subject == \'Math\') as correct_math' \
    ' from default.peas' \
    ' group by st_id' \
    ' ) as count_peas' \
    ' on default.studs.st_id = count_peas.st_id' \
    ' left join default.final_project_check' \
    ' on default.studs.st_id = default.final_project_check.st_id' \
    ' )' \
    ' select  test_grp,' \
    ' round(sum(money) / count(distinct st_id), 0) as ARPU,' \
    ' round(sum(money) / count(distinct st_id) filter (where correct_all > 10), 0) as ARPAU,' \
    ' round(count(distinct st_id) filter (where money > 0) / count(distinct st_id), 2) as CR,' \
    ' round(count(distinct st_id) filter (where money > 0) / count(distinct st_id) ' \
    ' filter (where correct_all > 10), 2) as CR_active,' \
    ' round(count(distinct st_id) filter (where money > 0 and subject == \'Math\') / count(distinct st_id) ' \
    ' filter (where correct_math >= 2), 2) as CR_math' \
    ' from all_data' \
    ' group by test_grp' 

In [8]:
df_task2 = ph.read_clickhouse(query_task2, connection=connection)
df_task2.head()

,test_grp,ARPU,ARPAU,CR,CR_active,CR_math
0,control,4541.0,10906.0,0.05,0.12,0.16
1,pilot,11508.0,35365.0,0.11,0.33,0.38


***

### 2.2 Оптимизация воронки
### Корректировка

Добавила фильтр в обе части уравнения

In [9]:
query_task2_corr = ' with all_data as' \
    ' (' \
    ' select  default.studs.st_id as st_id,' \
    ' test_grp,' \
    ' correct_all,' \
    ' correct_math,' \
    ' subject,' \
    ' sale_time,' \
    ' money' \
    ' from default.studs' \
    ' left join' \
    ' (' \
    ' select  st_id,' \
    ' sum(correct) as correct_all,' \
    ' sum(correct) filter (where subject == \'Math\') as correct_math' \
    ' from default.peas' \
    ' group by st_id' \
    ' ) as count_peas' \
    ' on default.studs.st_id = count_peas.st_id' \
    ' left join default.final_project_check' \
    ' on default.studs.st_id = default.final_project_check.st_id' \
    ' )' \
    ' select  test_grp,' \
    ' round(sum(money) / count(distinct st_id), 0) as ARPU,' \
    ' round(sum(money) filter (where correct_all > 10) / ' \
    ' count(distinct st_id) filter (where correct_all > 10), 0) as ARPAU,' \
    ' round(count(distinct st_id) filter (where money > 0) / count(distinct st_id), 2) as CR,' \
    ' round(count(distinct st_id) filter (where money > 0 and correct_all > 10) / count(distinct st_id) ' \
    ' filter (where correct_all > 10), 2) as CR_active,' \
    ' round(count(distinct st_id) filter (where money > 0 and subject == \'Math\' and correct_math >= 2) ' \
    ' / count(distinct st_id) ' \
    ' filter (where correct_math >= 2), 2) as CR_math' \
    ' from all_data' \
    ' group by test_grp' 

In [10]:
df_task2_corr = ph.read_clickhouse(query_task2_corr, connection=connection)
df_task2_corr.head()

,test_grp,ARPU,ARPAU,CR,CR_active,CR_math
0,control,4541.0,10394.0,0.05,0.11,0.06
1,pilot,11508.0,29740.0,0.11,0.26,0.10


***

### 2.1 Очень усердные ученики. Код запроса, чтобы вставить в Clickhouse

with max_date as   
( select toStartOfMonth(max(timest)) as max_date from default.peas )   
   
select count(st_id) as count_best_students from    
(   
    select st_id, sum(correct) as count_correct_peas  from default.peas  
    where toStartOfMonth(timest) = (select max_date from max_date)  
    group by st_id   
    having count_correct_peas >= 20   
) as best_students   

### 2.2 Оптимизация воронки. Код запроса, чтобы вставить в Clickhouse

with all_data as (  
    select  default.studs.st_id as st_id,   
            test_grp,   
            correct_all,   
            correct_math,   
            subject,   
            sale_time,   
            money  
    from default.studs   
    left join   
    (   
        select  st_id,   
                sum(correct) as correct_all,   
                sum(correct) filter (where subject == 'Math') as correct_math   
        from default.peas   
        group by st_id   
    ) as count_peas   
    on default.studs.st_id = count_peas.st_id   
    left join default.final_project_check   
    on default.studs.st_id = default.final_project_check.st_id   
)   
   
select  test_grp,   
        round(sum(money) / count(distinct st_id), 0) as ARPU,   
        round(sum(money) / count(distinct st_id) filter (where correct_all > 10), 0) as ARPAU,   
        round(count(distinct st_id) filter (where money > 0) / count(distinct st_id), 2) as CR,   
        round(count(distinct st_id) filter (where money > 0) / count(distinct st_id) filter (where correct_all > 10), 2) as CR_active,   
        round(count(distinct st_id) filter (where money > 0 and subject == 'Math') / count(distinct st_id) filter (where correct_math >= 2), 2) as CR_math   
from all_data   
group by test_grp   

### 2.2 Оптимизация воронки. Код запроса, чтобы вставить в Clickhouse
### Корректировка

with all_data as (  
    select  default.studs.st_id as st_id,   
            test_grp,   
            correct_all, 
            correct_math,   
            subject,   
            sale_time,   
            money  
    from default.studs   
    left join   
    (   
        select  st_id,   
                sum(correct) as correct_all,   
                sum(correct) filter (where subject == 'Math') as correct_math   
        from default.peas   
        group by st_id   
    ) as count_peas   
    on default.studs.st_id = count_peas.st_id
    left join default.final_project_check   
    on default.studs.st_id = default.final_project_check.st_id   
)
   
select  test_grp,   
        round(sum(money) / count(distinct st_id), 0) as ARPU,   
        round(sum(money) filter (where correct_all > 10) / count(distinct st_id) filter (where correct_all > 10), 0) as ARPAU,    
        round(count(distinct st_id) filter (where money > 0) / count(distinct st_id), 2) as CR,   
        round(count(distinct st_id) filter (where money > 0 and where correct_all > 10) / count(distinct st_id) filter (where correct_all > 10), 2) as CR_active,    
        round(count(distinct st_id) filter (where money > 0 and subject == 'Math' and correct_math >= 2) / count(distinct st_id) filter (where correct_math >= 2), 2) as CR_math   
from all_data   
group by test_grp